In [1]:
import sagemaker
from sagemaker.huggingface import HuggingFace

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [2]:
role = sagemaker.get_execution_role()

In [3]:
role

'arn:aws:iam::445792334809:role/SageMakerLLMRole'

In [10]:
hyperparameters= {
    "model_id": "TinyLlama/TinyLlama-1.1B-intermediate-step-1431k-3T",
    'epochs': 2, 
    'per_device_train_batch_size': 2,
    'lr': 2e-5
}

In [41]:
estimator = HuggingFace(
    entry_point= "train.py",
    source_dir = './scripts',
    role= role,
    transformers_version= '4.36',
    pytorch_version= '2.1',
    py_version='py310',
    instance_type= 'ml.g5.xlarge',
    instance_count= 1,
    output_path="s3://llm-model-artifacts-sayed/models/",
    hyperparameters= hyperparameters
    
)

In [42]:
estimator.fit({
    'train': 's3://llm-finetune-dataset-ashfaq/datasets/'
})

INFO:sagemaker.telemetry.telemetry_logging:SageMaker Python SDK will collect telemetry to help us better understand our user's needs, diagnose issues, and deliver additional features.
To opt out of telemetry, please disable via TelemetryOptOut parameter in SDK defaults config. For more information, refer to https://sagemaker.readthedocs.io/en/stable/overview.html#configuring-and-using-defaults-with-the-sagemaker-python-sdk.
INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker:Creating training-job with name: huggingface-pytorch-training-2025-11-27-12-31-33-882


2025-11-27 12:31:34 Starting - Starting the training job...
2025-11-27 12:31:58 Pending - Training job waiting for capacity...
2025-11-27 12:32:35 Downloading - Downloading input data...
2025-11-27 12:33:00 Downloading - Downloading the training image........................
2025-11-27 12:36:53 Training - Training image download completed. Training in progress.bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
/opt/conda/lib/python3.10/site-packages/paramiko/pkey.py:100: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "cipher": algorithms.TripleDES,
/opt/conda/lib/python3.10/site-packages/paramiko/transport.py:259: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "class": algorithms.TripleDE

In [43]:
estimator.model_data

's3://llm-model-artifacts-sayed/models/huggingface-pytorch-training-2025-11-27-12-31-33-882/output/model.tar.gz'

## Deploying the model to use

In [59]:
import sagemaker
from sagemaker.huggingface import HuggingFaceModel

role = sagemaker.get_execution_role()

model = HuggingFaceModel(
    model_data = "s3://llm-model-artifacts-sayed/models/huggingface-pytorch-training-2025-11-27-12-31-33-882/output/model.tar.gz",
    role= role,
    transformers_version= "4.37.0",
    pytorch_version= '2.1.0',
    py_version= "py310",
    env= {
        "HF_TASK": "text-generation"
    }
)

predictor = model.deploy(
    initial_instance_count= 1, 
    instance_type= "ml.g5.xlarge", # change from ml.m5.xlarge to ml.g5.xlarge
    endpoint_name= "customer-support-finetuned-llm-live-endpoint"
)

INFO:sagemaker:Creating model with name: huggingface-pytorch-inference-2025-11-27-13-37-20-028
INFO:sagemaker:Creating endpoint-config with name customer-support-finetuned-llm-live-endpoint
INFO:sagemaker:Creating endpoint with name customer-support-finetuned-llm-live-endpoint


---------!

In [78]:
## THIS IS JUST TO VALIDATE WHETHER MODEL WORKING OR NOT IN THE NOTEBOOK ITSELF
response = predictor.predict({
    "inputs": "I want to cancel my order, what should i do",
    "parameters": {
        "max_new_tokens": 50,
        "temperature": 0.7,
    }
})
print(response)

[{'generated_text': 'I want to cancel my order, what should i do?\nYou can contact our customer service team at 1-800-888-2287 or email us at support@sportsmart.com.\nHow do I know if my order is shipped?'}]


In [82]:
## THIS IS JUST TO VALIDATE WHETHER MODEL WORKING OR NOT IN THE NOTEBOOK ITSELF
response = predictor.predict({
    "inputs": "User: i want refund, what should i do",
    "parameters": {
        "max_new_tokens": 50,
        "temperature": 0.1,
        "do_sample": False
    }
})
print(response)

[{'generated_text': 'User: i want refund, what should i do?\n\nA: You can use the following code to get the refund status.\n$refund_status = $this->api_model->get_refund_status($refund_id);\n\n$refund_'}]


In [86]:
## THIS IS JUST TO VALIDATE WHETHER MODEL WORKING OR NOT IN THE NOTEBOOK ITSELF
response = predictor.predict({
    "inputs": "User: what can you do",
    "parameters": {
        "max_new_tokens": 60,
        "temperature": 0.1,
        "do_sample": False
    }
})
print(response)

[{'generated_text': 'User: what can you do to help?\nI\'m not sure if this is the right place to ask this, but I\'m not sure if it\'s a bug or not.\nI\'m using the latest version of the plugin, and I\'m trying to use the "Add to Cart" button'}]
